In [61]:
#join data with risk
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tabulate import tabulate
from termcolor import colored

In [66]:
class RiskPrediction():
    def __init__(self):
        self.data=None
        self.column_list=None
        self.assert_did=None
        self.risk=None
        
    def load_data(self):
        data=pd.read_csv('AuditAreaData.csv')
        data1=pd.read_csv('rawdata/e_aa_asserts.csv')
        result=data1.ix[:,['eng_aa_sid','assert_did','inherent_risk_did','ctrlrisk_did','combined_risk_did']].merge(data,on='eng_aa_sid')

        # replace missing value with 'None'
        column_ls=result.columns.tolist()[5:]
        self.column_ls=column_ls
        result[column_ls]=result[column_ls].fillna('None')

        #remove testing data
        for column in column_ls:
            result=result[result[column].str.contains("test") == False]
            result=result[result[column].str.contains("Test") == False]

        print('after clearning the testing data,  %s rows left' % (result.shape[0]))
        print(result['assert_did'].value_counts())
        self.data=result
        
    def create(self,assert_did,risk):
        self.assert_did=assert_did
        self.risk=risk
        
    def build(self):
        stop_list=self.stop_words()
        df=self.data_preprocessing()

        #get training data
        train=pd.DataFrame()
        for index, column in enumerate(self.column_ls):
            X=df[column]
            cvec=CountVectorizer(max_features=20,lowercase=True, stop_words= stop_list).fit(X)
            temp=pd.DataFrame(cvec.transform(X).todense(),columns=[ str(index)+'_'+value for value in cvec.get_feature_names()])
            train=pd.concat([train,temp],axis=1)
        print('training shape is', train.shape)

        # get accuracy
        model=RandomForestClassifier()
        result=cross_val_score(model, train, df[self.risk].tolist(), cv=6, scoring = 'accuracy')
        print(colored('accuracy rate of the model is: %s' % (result.mean()),'red'),'\n')

        #build model    
        rf = RandomForestClassifier()
        rf.fit(train, df[self.risk].tolist())

        #analysis
        headers = ["name", "score"]
        values = sorted(zip(train.columns.tolist(), rf.feature_importances_), key=lambda x: x[1] * -1)
        values = values[0:15] #top 15
        print(tabulate(values, headers, tablefmt="plain"))
        
#-------------------------------------------
    def stop_words(self):
        stop_list=pd.read_csv('/Users/katherine/nltk_data/corpora/stopwords/english',header=None)[0].tolist()
        stop_list.remove('no')
        stop_list.remove('not')
        return stop_list

    def data_preprocessing(self):
        df=self.data[self.data.assert_did==self.assert_did]
        row1=df.shape[0]
        df=df.dropna(subset=[self.risk])
        print('there are %s missing rows, so only %s rows left' % (row1-df.shape[0],df.shape[0]),'\n')
        print(df[self.risk].value_counts(),'\n')
        print(colored('baseline accuracy rate: %s' % (df[self.risk].value_counts().reset_index().ix[0,1]/df.shape[0]),'red'),'\n')
        return df


In [67]:
model=RiskPrediction()
model.load_data()

after clearning the testing data,  15375 rows left
3    2563
2    2563
1    2563
7    2560
6    2560
8    2560
5       3
4       3
Name: assert_did, dtype: int64


In [68]:
model.create(assert_did=3,risk='combined_risk_did') # 'inherent_risk_did', 'ctrlrisk_did', 'combined_risk_did'

In [69]:
model.build()

there are 2287 missing rows, so only 276 rows left 

1.0    220
2.0     50
4.0      6
Name: combined_risk_did, dtype: int64 

baseline accuracy rate: 0.797101449275 

training shape is (276, 632)
accuracy rate of the model is: 0.789947579402 

name               score
13_npo         0.0445269
5_fees         0.035908
23_management  0.0348131
11_basis       0.0302107
3_controls     0.0268968
0_would        0.0234855
33_company     0.0179181
24_odh         0.0178812
23_revenue     0.0174884
33_no          0.017053
3_could        0.0169313
22_revenue     0.0165524
0_year         0.0161978
7_management   0.0151684
7_not          0.0143692
